In [1]:
import torch
import torchvision as vision
from torch.utils.data import DataLoader, TensorDataset
from sklearn.cluster import DBSCAN
from torch import nn

In [2]:
device = torch.device('cuda')

In [3]:
data = torch.load("data/processed/mini_dataset.pt", device)

In [4]:
data.shape

torch.Size([5000, 60483])

In [5]:
data_y = torch.load("data/processed/mini_data_y.pt", device)

In [6]:
data_set = TensorDataset(data.float(), data_y.long())

In [13]:
train_batches = DataLoader(data_set, batch_size=500, shuffle=True)

In [16]:
sum(data.var(dim=0) < 1e-2)

tensor(17300, device='cuda:0')

In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
#         self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(60483, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
#         x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [11]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=60483, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [24]:
logits = model(data.float())
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")


Predicted class: tensor([3, 3, 3,  ..., 3, 3, 3], device='cuda:0')


In [29]:

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 60483]) | Values : tensor([[ 0.0040,  0.0016, -0.0030,  ...,  0.0001, -0.0010,  0.0033],
        [ 0.0008,  0.0025,  0.0029,  ...,  0.0037,  0.0017,  0.0003]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0013, -0.0021], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0060, -0.0316, -0.0224,  ..., -0.0084,  0.0358,  0.0067],
        [ 0.0336, -0.0095,  0.0209,  ...,  0.0296,  0.0317, -0.0426]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.bias | Size: torch.Size([512]) | Values : tensor([0.0421, 0.0391], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.4.weight | Size: torch.Size([10, 512]) | Values : tensor([[-0.0198, -0.0364, -0.0046,  ..., -0.0296, -0.0424,  0.0372],
        [ 0.0357, -0.0122, -0.0193,  ..

In [9]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [14]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_batches, model, loss_fn, optimizer)
#     test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.294865  [    0/ 5000]
Epoch 2
-------------------------------
loss:     inf  [    0/ 5000]
Epoch 3
-------------------------------
loss: 2.302585  [    0/ 5000]
Epoch 4
-------------------------------
loss: 2.302585  [    0/ 5000]
Epoch 5
-------------------------------
loss: 2.293375  [    0/ 5000]
Epoch 6
-------------------------------
loss: 2.297980  [    0/ 5000]
Epoch 7
-------------------------------
loss: 2.302585  [    0/ 5000]
Epoch 8
-------------------------------
loss: 2.302585  [    0/ 5000]
Epoch 9
-------------------------------
loss: 2.288769  [    0/ 5000]
Epoch 10
-------------------------------
loss: 2.293375  [    0/ 5000]
Done!
